In [7]:
#!pip install openpyxl --upgrade --pre

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.1/242.1 kB 1.2 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: openpyxl
    Found existing installation: openpyxl 3.0.5
    Uninstalling openpyxl-3.0.5:
      Successfully uninstalled openpyxl-3.0.5

[notice] A new release of pip available: 22.2.2 -> 22.3
[notice] To update, run: pip install --upgrade pip


In [3]:
import os
import regex as re
import emoji
import pandas as pd
import numpy as np

from collections import Counter, defaultdict
from nltk.corpus import stopwords
from string import punctuation

sw = stopwords.words("english")
import openpyxl

In [4]:
# Add any additional import statements you need here
import html
from textacy import preprocessing
import string
import nltk

stop_words = set(stopwords.words("english"))

In [2]:
df = pd.read_excel('reddit-data.xlsx')
df

,Unnamed: 0,subreddit,text
0,0,personalfinance,"# Welcome! Before making a post, please check..."
1,1,personalfinance,"### If you need help, please check the [PF Wik..."
2,2,personalfinance,Haven't been churning so much lately; but foun...
3,3,personalfinance,**TL;DR** my question is: If I set up a 529 Sa...
4,4,personalfinance,"I've lived in my house in VA since 2009, and a..."
...,...,...,...
4768,895,story,Do you ever make up stories that are so absurd...
4769,896,story,"So up until about 2019, I was unable to watch ..."
4770,897,story,I had to shit at the airport (I couldn't shit ...
4771,898,story,"This morning as I was grabbing my right shoe, ..."


In [9]:
print(df['text'])

0       # Welcome!  Before making a post, please check...
1       ### If you need help, please check the [PF Wik...
2       Haven't been churning so much lately; but foun...
3       **TL;DR** my question is: If I set up a 529 Sa...
4       I've lived in my house in VA since 2009, and a...
                              ...                        
4768    Do you ever make up stories that are so absurd...
4769    So up until about 2019, I was unable to watch ...
4770    I had to shit at the airport (I couldn't shit ...
4771    This morning as I was grabbing my right shoe, ...
4772         **True peace is only won by peaceful means**
Name: text, Length: 4773, dtype: object


In [5]:
def descriptive_stats(tokens, num_tokens = 5, verbose=True) :
    """
        Given a list of tokens, print number of tokens, number of unique tokens, 
        number of characters, lexical diversity (https://en.wikipedia.org/wiki/Lexical_diversity), 
        and num_tokens most common tokens. Return a list with the number of tokens, number
        of unique tokens, lexical diversity, and number of characters. 
    
    """

    # Fill in the correct values here. 
    num_tokens = len(tokens)
    num_unique_tokens = len(set(tokens))
    lexical_diversity = num_unique_tokens/num_tokens
    num_characters = 0
    for i in tokens:
        num_characters += len(i)
    if verbose :        
        print(f"There are {num_tokens} tokens in the data.")
        print(f"There are {num_unique_tokens} unique tokens in the data.")
        print(f"There are {num_characters} characters in the data.")
        print(f"The lexical diversity is {lexical_diversity:.3f} in the data.")
    
        # print the five most common tokens
        
    return([num_tokens, num_unique_tokens,
            lexical_diversity,
            num_characters])
    

In [6]:
text = "here is some example text with other example text here in this text".split()
assert(descriptive_stats(text, verbose=True)[0] == 13)
assert(descriptive_stats(text, verbose=False)[1] == 9)
assert(abs(descriptive_stats(text, verbose=False)[2] - 0.69) < 0.02)
assert(descriptive_stats(text, verbose=False)[3] == 55)


There are 13 tokens in the data.
There are 9 unique tokens in the data.
There are 55 characters in the data.
The lexical diversity is 0.692 in the data.


In [7]:
# create your clean twitter data here

## This calculates the impurity in an instance of tokens, used to find characters worth removing
RE_SUSPICIOUS = re.compile(r'[&#<>{}\[\]\\]')
def impurity(text, min_len=10):
    """returns the share of suspicious characters in a text""" 
    if text == None or len(text) < min_len:
        return 0 
    else:
        return len(RE_SUSPICIOUS.findall(text))/len(text)
    print(impurity(text))

## Function to clean the text by removing matches on the regex rules
def clean(text):
    # convert html escapes like &amp; to characters.
    text = html.unescape(text)
    # tags like <tab>
    text = re.sub(r'<[^<>]*>', ' ', text)
    # markdown URLs like [Some text](https://....)
    text = re.sub(r'\[([^\[\]]*)\]\([^\(\)]*\)', r'\1', text)
    # text or code in brackets like [0]
    text = re.sub(r'\[[^\[\]]*\]', ' ', text)
    # standalone sequences of specials, matches &# but not #cool
    text = re.sub(r'(?:^|\s)[&#<>{}\[\]+|\\:-]{1,}(?:\s|$)', ' ', text) # standalone sequences of hyphens like --- or ==
    text = re.sub(r'(?:^|\s)[\-=\+]{2,}(?:\s|$)', ' ', text)
    # sequences of white spaces
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'##',' ',text)
    text = re.sub(r'{{_}}',' ',text)
    text = re.sub(r'/\/',' ',text)
    text = re.sub(r'//\\',' ',text)
    text = re.sub(r'\\',' ',text)
    #Remove punctuation
    text = re.sub(r'[^\w\s]', ' ', text)
    return text.strip()

def normalize(text):
    text = preprocessing.normalize.hyphenated_words(text)
    text = preprocessing.normalize.quotation_marks(text)
    text = preprocessing.normalize.unicode(text)
    text = preprocessing.remove.accents(text)
    return text


def tokenize(text):
    return re.findall(r'[\w-]*\p{L}[\w-]*', text)

def remove_stop(tokens):
    return [t for t in tokens if t.lower() not in stop_words]

pipeline = [str.lower, tokenize, remove_stop]

def prepare(text, pipeline): 
    tokens = text
    for transform in pipeline:
        tokens = transform(tokens)
    return tokens

In [10]:
df['text'] = df['text'].astype('string')

In [12]:
df['text'] = df['text'].fillna('')

In [13]:
df['clean_text'] = df['text'].map(clean)
df['clean_text'] = df['clean_text'].astype('string')
df['clean_text'] = df['clean_text'].map(normalize)
df['tokens'] = df['clean_text'].apply(prepare, pipeline=pipeline)
df.head()

,Unnamed: 0,subreddit,text,clean_text,tokens
0,0,personalfinance,"# Welcome! Before making a post, please check...",Welcome Before making a post please check ou...,"[welcome, making, post, please, check, great, ..."
1,1,personalfinance,"### If you need help, please check the [PF Wik...",If you need help please check the PF Wiki to ...,"[need, help, please, check, pf, wiki, see, que..."
2,2,personalfinance,Haven't been churning so much lately; but foun...,Haven t been churning so much lately but foun...,"[churning, much, lately, found, make, mortgage..."
3,3,personalfinance,**TL;DR** my question is: If I set up a 529 Sa...,TL DR my question is If I set up a 529 Savi...,"[tl, dr, question, set, savings, account, niec..."
4,4,personalfinance,"I've lived in my house in VA since 2009, and a...",I ve lived in my house in VA since 2009 and a...,"[lived, house, va, since, along, years, house,..."


In [14]:
posts = []
for description in df["clean_text"]:
    posts.extend(description.split())

In [15]:
print('Subreddit Descriptive Statistics')
print(descriptive_stats(posts))

Subreddit Descriptive Statistics
There are 1042337 tokens in the data.
There are 35401 unique tokens in the data.
There are 4201232 characters in the data.
The lexical diversity is 0.034 in the data.
[1042337, 35401, 0.0339631040632732, 4201232]
